<a href="https://colab.research.google.com/github/taylorcmichel/thisismystuff/blob/main/Loyalists_Polars_with_Names.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import glob
import polars as pl

In [2]:
path = '/content/drive/MyDrive/Colab Notebooks/'
datapath = 'Loyalist Project/Membership Data Files/CSVs/'
files = '* Membership Data.csv'
fullpath = path + datapath + files
allfiles = []

In [3]:
for file in sorted(glob.glob(fullpath)):
  q = pl.scan_csv(file, infer_schema_length=int(10e10),
                  truncate_ragged_lines=True)
  allfiles.append(q)

In [4]:
full_list_df = (
    pl.read_csv(path + datapath + '2013-2024 Individual Members.csv',
                infer_schema_length=0)
    .with_columns(pl.all().cast(pl.Utf8, strict=False))
)

In [5]:
dfs = pl.collect_all(allfiles)

In [6]:
def cleanandstrip(dataframe, allfiles):
  for x in range(len(allfiles)):
    dataframe[x] = dataframe[x].filter(
        ~pl.col('Primary Member Type').is_in(['Emeritus',
                                              'Travel Fellow',
                                              'Honorary'])
    )
    dataframe[x] = dataframe[x][[
        'individual_id'
    ]]
    dataframe[x] = dataframe[x].rename(
        {
            'individual_id': 'Contact Number'
        }
    )
    dataframe[x] = dataframe[x].select(
        pl.col('Contact Number').cast(pl.Utf8)
    )

In [7]:
cleanandstrip(dfs, allfiles)

In [8]:
main_all_df = (
    pl.concat(
        [dfs[0], dfs[1], dfs[2], dfs[3], dfs[4], dfs[5]], how='align'
    )
)

In [9]:
all_mems = (
    main_all_df.clone()
    .unique(
        subset='Contact Number', maintain_order=True
    )
)

In [10]:
years = ['_2018', '_2019', '_2020', '_2021', '_2022', '_2023']

new_build_on_df = all_mems[['Contact Number']]

for x in range(len(allfiles)):
  new_build_on_df = (
      new_build_on_df.join(
          dfs[x], on='Contact Number', suffix=years[x], how='outer'
      )
      .unique(
          subset='Contact Number', maintain_order=True
      )
  )
  new_build_on_df = (
      new_build_on_df.with_columns(
        pl.col(
            f'Contact Number{years[x]}'
        )
        .is_not_null()
      )
  )

In [11]:
total_true = (
    new_build_on_df.with_columns(
        sum=pl.sum_horizontal('Contact Number_2018',
                              'Contact Number_2019',
                              'Contact Number_2020',
                              'Contact Number_2021',
                              'Contact Number_2022',
                              'Contact Number_2023')
        )
)

In [12]:
total_true = (
    total_true.with_columns(
        pl.when(
            pl.col('sum') >= 5
        )
        .then(
            pl.lit('Y')
        )
        .otherwise(
            pl.lit('N')
        )
        .alias('Loyalist')
    )
)

In [13]:
total_loyalists = (
    total_true.select(
        [
            pl.col('Loyalist').value_counts(sort=True)
        ]
    )
)

In [14]:
loyalist_df = (
    total_true.filter(
        pl.col('Loyalist') == 'Y'
    )
)

In [15]:
loyalist_df = (
    loyalist_df.join(
        full_list_df, left_on='Contact Number',
        right_on='pa_contactnumber_stripped',
        suffix='_w/ NAMES',
        how='left')
).unique(subset='Contact Number', maintain_order=True)

In [16]:
final_df = (
    loyalist_df.select(
        pl.col('Contact Number_w/ NAMES'),
        pl.col('Full Name'),
        pl.col('Contact Number_2018'),
        pl.col('Contact Number_2019'),
        pl.col('Contact Number_2020'),
        pl.col('Contact Number_2021'),
        pl.col('Contact Number_2022'),
        pl.col('Contact Number_2023'),
        pl.col('sum'),
        pl.col('Loyalist')
    )
)

print(final_df)
print(total_loyalists)

shape: (11_513, 10)
┌────────────┬────────────┬────────────┬────────────┬───┬────────────┬────────────┬─────┬──────────┐
│ Contact    ┆ Full Name  ┆ Contact    ┆ Contact    ┆ … ┆ Contact    ┆ Contact    ┆ sum ┆ Loyalist │
│ Number_w/  ┆ ---        ┆ Number_201 ┆ Number_201 ┆   ┆ Number_202 ┆ Number_202 ┆ --- ┆ ---      │
│ NAMES      ┆ str        ┆ 8          ┆ 9          ┆   ┆ 2          ┆ 3          ┆ u32 ┆ str      │
│ ---        ┆            ┆ ---        ┆ ---        ┆   ┆ ---        ┆ ---        ┆     ┆          │
│ str        ┆            ┆ bool       ┆ bool       ┆   ┆ bool       ┆ bool       ┆     ┆          │
╞════════════╪════════════╪════════════╪════════════╪═══╪════════════╪════════════╪═════╪══════════╡
│ 000001000  ┆ Kaczmarek, ┆ true       ┆ true       ┆ … ┆ true       ┆ true       ┆ 6   ┆ Y        │
│            ┆ Leonard K  ┆            ┆            ┆   ┆            ┆            ┆     ┆          │
│ 000010000  ┆ Desimone,  ┆ true       ┆ true       ┆ … ┆ true       ┆ 